Нотбук для перевода сгенрирванного датасета через stable diffusion в датасет YOLO для детекции

In [21]:
from PIL import Image
import os
import pandas as pd
import glob
import shutil
from tqdm.auto import tqdm
from utility import check_yolo_label, parse_xml

In [22]:
IMAGE_INPUT_DIR = r"C:\project\Computer_Vision\term_paper_2023\записи\26.02\Описание Anti_UAV_aug_stable_diffusion\Anti_UAV_aug_stable_diffusion"
LABELS_INPUT_DIR = r"C:\project\Computer_Vision\term_paper_2023\datasets\antiuav_train\xml"

def get_dron_df():
    df = pd.DataFrame (glob.glob( os.path.join(IMAGE_INPUT_DIR,'*.jpg')), columns=['path_image'])
    df['id_version'] = df['path_image'].apply(lambda x : x.split('\\')[-1].split('.')[0])
    df['version'] = df['id_version'].apply(lambda x : x.split('_')[1])
    df['id'] = df['id_version'].apply(lambda x : x.split('_')[0])
    df['path_label'] = df['id'].apply(lambda x :  os.path.join(LABELS_INPUT_DIR,f"{x}.xml"))
    return df

In [23]:
OUTPUT_DIR = r"C:\project\Computer_Vision\term_paper_2023\записи\26.02\Описание Anti_UAV_aug_stable_diffusion\Anti_UAV_aug_stable_diffusion_yolo"
IMAGE_OUTPUT_DIR = os.path.join(OUTPUT_DIR,"images" )
LABEL_OUTPUT_DIR = os.path.join(OUTPUT_DIR,"labels" )
if os.path.exists(OUTPUT_DIR):
    shutil.rmtree(OUTPUT_DIR)

os.makedirs(os.path.join(IMAGE_OUTPUT_DIR))
os.makedirs(os.path.join(LABEL_OUTPUT_DIR))

df_drons = get_dron_df()

In [24]:
def dataset2yolo(df):
    for i, row in tqdm(df.iterrows()):
        coords_drons , width_image, height_image = parse_xml(row['path_label'])
        str_label_yolo = ""

        xmin, ymin, xmax, ymax  = coords_drons[0]

        width_drone = (xmax - xmin)
        height_drone = (ymax - ymin)
        width_drone_yolo = width_drone / width_image
        height_drone_yolo = height_drone / height_image

        x_center_yolo = (xmin + width_drone / 2) / width_image
        y_center_yolo = (ymin + height_drone / 2) / height_image

        str_label_yolo += f"0 {x_center_yolo} {y_center_yolo} {width_drone_yolo} {height_drone_yolo} \n"

        #CHECK LABEL
        if i<5:
            check_yolo_label(Image.open(row['path_image']), x_center_yolo, y_center_yolo, width_drone_yolo, height_drone_yolo )

        path_label_output = os.path.join(LABEL_OUTPUT_DIR, row['id_version'] + '.txt')
        image_save_path = os.path.join(IMAGE_OUTPUT_DIR, f"{row['id_version']}.jpg")

        with open(path_label_output, 'w') as f:
            f.write(str_label_yolo)

        Image.open(row['path_image']).resize((512,512)).save(image_save_path)

In [25]:
dataset2yolo(df_drons)

0it [00:00, ?it/s]

In [26]:
shutil.make_archive(OUTPUT_DIR, 'zip', OUTPUT_DIR)

'C:\\project\\Computer_Vision\\term_paper_2023\\записи\\26.02\\Описание Anti_UAV_aug_stable_diffusion\\Anti_UAV_aug_stable_diffusion_yolo.zip'